In [16]:
from mqt.bench import get_benchmark
from mqt.bench import CompilerSettings, QiskitSettings, TKETSettings, get_benchmark
from qiskit import *
import numpy as np

In [17]:
n=3

In [18]:
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
qc = get_benchmark(benchmark_name="dj",
                    level="nativegates",
                    circuit_size=n,
                    compiler="qiskit",
                    compiler_settings=compiler_settings,
                    provider_name="ionq",)
print("---------------dj---------------")
print(qc.draw())
print("=================================================================================================")


---------------dj---------------
global phase: 3π/2
      ┌───────┐  ┌───────────┐ ┌────────┐  ┌──────────┐                        »
q_0: ─┤ Ry(π) ├──┤0          ├─┤ Rz(-π) ├──┤ Rx(-π/2) ├────────────────────────»
      ├───────┤  │           │ └────────┘ ┌┴──────────┤ ┌────────┐ ┌──────────┐»
q_1: ─┤ Ry(π) ├──┤  Rxx(π/2) ├────────────┤0          ├─┤ Rz(-π) ├─┤ Rx(-π/2) ├»
     ┌┴───────┴─┐│           │┌──────────┐│  Rxx(π/2) │┌┴────────┴┐└──────────┘»
q_2: ┤ Ry(-π/2) ├┤1          ├┤ Rx(-π/2) ├┤1          ├┤ Rx(-π/2) ├────────────»
     └──────────┘└───────────┘└──────────┘└───────────┘└──────────┘            »
c: 2/══════════════════════════════════════════════════════════════════════════»
                                                                               »
«      ░ ┌─┐   
«q_0: ─░─┤M├───
«      ░ └╥┘┌─┐
«q_1: ─░──╫─┤M├
«      ░  ║ └╥┘
«q_2: ─░──╫──╫─
«      ░  ║  ║ 
«c: 2/════╩══╩═
«         0  1 


In [20]:
from qiskit_aer import AerSimulator
from qiskit import *
from qiskit.providers.fake_provider import *
import numpy as np 
import os
from qiskit.primitives import StatevectorSampler
sampler = StatevectorSampler()
job = sampler.run([qc], shots=1000)
result = job.result()
count = result[0].data["meas"].get_counts()
#print(f" > Counts: {result[0].data["meas"].get_counts()}")

KeyError: 'Key (meas) does not exist in this data bin.'

In [15]:
result

PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=1000, num_bits=2>)), metadata={'shots': 1000, 'circuit_metadata': {}})], metadata={'version': 2})

In [21]:
result[0]

SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=1000, num_bits=2>)), metadata={'shots': 1000, 'circuit_metadata': {}})

In [23]:
result[0].data

DataBin(c=BitArray(<shape=(), num_shots=1000, num_bits=2>))

In [31]:
data

DataBin(c=BitArray(<shape=(), num_shots=1000, num_bits=2>))

In [34]:
data1= result[0].data
data1

DataBin(c=BitArray(<shape=(), num_shots=1000, num_bits=2>))

In [33]:
data

DataBin(c=BitArray(<shape=(), num_shots=1000, num_bits=2>))

In [5]:
if type == 0: key = '0'*input_size
    else: key = '1'*input_size
correct_dist={key: 1,0}

IndentationError: unexpected indent (2656517398.py, line 2)

In [ ]:
def hellinger_fidelity_with_expected(p=count, q={key: 1.0}):
    p_sum = sum(p.values())
    q_sum = sum(q.values())
    if q_sum == 0:
        print("ERROR: polarization_fidelity(), expected distribution is invalid, all counts equal to 0")
        return 0
    p_normed = {}
    for key, val in p.items():
        p_normed[key] = val/p_sum
    q_normed = {}
    for key, val in q.items():
        q_normed[key] = val/q_sum
    total = 0
    for key, val in p_normed.items():
        if key in q_normed.keys():
            total += (np.sqrt(val) - np.sqrt(q_normed[key]))**2
            del q_normed[key]
        else:
            total += val
    total += sum(q_normed.values())
    if total < 0:
        print("WARNING: using absolute value in fidelity calculation")
        total = abs(total)
        
    dist = np.sqrt(total)/np.sqrt(2)
    fidelity = (1-dist**2)**2

    return fidelity